In [6]:
import requests
import time
import re
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.kicktraq.com/archive/'

response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
page = response.text

In [ ]:
soup = BeautifulSoup(page,'lxml')

In [ ]:
kickstarter_page = []
funding = []
campaign_dates = []
backers = []
categories = []

all_projects_on_page = soup.find_all(class_="project first odd closed")
all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

for project in all_projects_on_page: 
    #projects.append(project)
    kickstarter_page.append(project.find('h2').find('a')['href'])
    funding.append(project.find(class_='project-details').find(text=re.compile('Funding: .\d* of .\d*.\d*')))
    campaign_dates.append(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)')))
    backers.append(project.find(class_='project-details').find(text=re.compile('Backers: \d*')))
    categories.append(project.find(class_='project-cat').find('a').text)

In [ ]:
funding[5].replace('$','')

In [ ]:
#projects[1]


In [76]:
projects=pd.DataFrame(columns=['kickstarter_page','funding', 'campaign dates', 'backers', 'category'])

In [77]:
#1-8000 are available
for page_num in range(1,2):
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    
    kickstarter_page = []
    funding = []
    campaign_dates = []
    backers = []
    categories = []
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    
    all_projects_on_page = soup.find_all(class_="project first odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

    for project in all_projects_on_page: 
        #projects.append(project)
        kickstarter_page.append(str(project.find('h2').find('a')['href']))
        funding.append(str(project.find(class_='project-details').find(text=re.compile('Funding: .\d* of .\d*.\d*')).replace('$','')))
        campaign_dates.append(str(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)'))))
        backers.append(str(project.find(class_='project-details').find(text=re.compile('Backers: \d*'))))
        categories.append(str(project.find(class_='project-cat').find('a').text))
        
        page_df = pd.DataFrame(np.column_stack([kickstarter_page,funding,campaign_dates,backers,categories]), 
                               columns=['kickstarter_page','funding', 'campaign dates', 'backers', 'category'])
        projects = pd.concat([projects, page_df])
        
    
    time.sleep(10+random.uniform(0,2))


ConnectionError: HTTPSConnectionPool(host='www.kicktraq.com', port=443): Max retries exceeded with url: /archive/?page=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f165927bd30>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [ ]:
projects.sample(10)

In [70]:
projects['description_length'] = 0
projects['n_pledges'] = 0
projects['pledge_level_0'] = 0
projects['pledge_level_1'] = 0
projects['pledge_level_2'] = 0
projects['pledge_level_3'] = 0
projects['pledge_level_4'] = 0
projects['pledge_level_5'] = 0
projects['pledge_level_6'] = 0
projects['pledge_level_7'] = 0
projects['pledge_level_8'] = 0
projects['pledge_level_9'] = 0
projects['pledge_level_10'] = 0
projects['pledge_level_11'] = 0
projects['pledge_level_12'] = 0
projects['pledge_level_13'] = 0
projects['pledge_level_14'] = 0
projects['pledge_level_15'] = 0
projects['pledge_level_16'] = 0
projects['pledge_level_17'] = 0
projects['pledge_level_18'] = 0
projects['pledge_level_19'] = 0
projects['pledge_level_20'] = 0

In [ ]:
for project in projects.head(5):
    url = 'https://www.kickstarter.com/' + project.kickstarter_page
        
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')

    pledges = soup.find_all(class_='pledge__info')
    
    n_pledges = len(pledges)
    
    pledge_amounts=[]
    for pledge in pledges:
        pledge_amounts.append(pledge.find(class_='money').text.replace('$',''))

    descript_paragraphs = soup.find(class_="full-description js-full-description responsive-media formatted-lists").find_all('p')
    description_words = 0
    for par in descript_paragraphs:
        description_words += len(str(par.text).split())
        
    project.description_length = description_words
    project.n_pledges = n_pledges
    for i, amt in enumerate(pledge_amounts):    
        project['pledge_level_'+str(i)] = int(amt[1:])    
        
    time.sleep(2+random.uniform(0,2))

In [74]:
for i, j in enumerate(['a','b','c']):
    print(i,j)

0 a
1 b
2 c


In [68]:
url = 'https://www.kickstarter.com//projects/1934089488/not-enough-fish/'

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page,'lxml')

pledges = soup.find_all(class_='pledge__info')
n_pledges = len(pledges)
pledge_amounts=[]
for pledge in pledges:
    pledge_amounts.append(pledge.find(class_='money').text.replace('$',''))

descript_paragraphs = soup.find(class_="full-description js-full-description responsive-media formatted-lists").find_all('p')
description_words = 0
for par in descript_paragraphs:
    description_words += len(str(par.text).split())

In [75]:
pledge_amounts

['£5', '£10', '£20', '£50', '£100', '£400', '£500']

In [59]:
descript_box

<div class="full-description js-full-description responsive-media formatted-lists">
<p>The recent changes to fishing quotas by the MMO (Marine Management Organisation)  in the UK, has led to incredible hardship for local fishermen around the country, but specifically in the South East and South West. The documentary will follow Pete and Chantelle (a fishing family) whom now struggle to make ends meet. Certain species such as Scape, Ray, and Plaice have been closed, with more species feared to join and the closures now set in place, families are struggling to pay there mortgages and put food on the table.</p><p>Our aim is to make an artistic, yet informative documentary to raise awareness of this issue and about how poor management has affect the livelihoods of many of British fisherman today. Although the news has given this story a certain level of coverage, we want to make a film that will reach out to a wider audience, conveying the true emotion behind this struggle, from an indepen